# Test Base Encoder

In [1]:
from BaseEncoder import BaseEncoder
from transformers import AutoTokenizer, TFAutoModel

In [2]:
sentence = 'Aku seorang kapiten'

In [3]:
base_encoder = BaseEncoder()

In [4]:
sample_tokenized = base_encoder.tokenize(['Aku', 'seorang', 'kapiten'])

In [5]:
sample_encoded = base_encoder.encode(['Aku', 'seorang', 'kapiten'])

In [6]:
sample_tokenized.shape

torch.Size([40])

# Data Loader

In [7]:
from DataLoader import read_examples_from_file, ReviewDataset

FILE_PATH = "dataset/test_2k_output_Feb_26_2020_output_Feb_18_2021.tsv"

In [8]:
test_data = ReviewDataset(FILE_PATH)

In [9]:
test_data[0]

(tensor([    3,  4557,  9790,    16,  5093,  8015,  1501,  4501,  4683,    18,
          1821, 14506,     1,  2677,  3868,    18,     4,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 tensor([[1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 1.],


In [10]:
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_data, batch_size=2, shuffle=True)

### Test to Batch

In [11]:
from SpanGenerator import SpanGenerator
span_generator = SpanGenerator()

In [12]:
X, y = next(iter(test_dataloader))

In [13]:
X_encoded = base_encoder.model(X)[0]

In [14]:
X_spans, X_span_ranges = span_generator.generate_spans(X_encoded)

In [15]:
X_spans.shape

torch.Size([2, 292, 8, 768])

# Test Model

In [16]:
from SpanMltri import SpanMltri

Using cuda device


In [17]:
span_mltri = SpanMltri()

In [19]:
term_scorer_logits = span_mltri(X)

In [21]:
term_scorer_logits[0]

tensor([[[0.0029, 0.0053, 0.0028, 0.0021, 0.0041],
         [0.0021, 0.0031, 0.0022, 0.0014, 0.0028],
         [0.0022, 0.0031, 0.0023, 0.0015, 0.0028],
         ...,
         [0.0026, 0.0034, 0.0027, 0.0020, 0.0032],
         [0.0028, 0.0034, 0.0029, 0.0024, 0.0033],
         [0.0031, 0.0034, 0.0031, 0.0028, 0.0034]],

        [[0.0008, 0.0029, 0.0008, 0.0008, 0.0033],
         [0.0021, 0.0031, 0.0022, 0.0014, 0.0028],
         [0.0022, 0.0031, 0.0023, 0.0015, 0.0028],
         ...,
         [0.0026, 0.0034, 0.0027, 0.0020, 0.0032],
         [0.0028, 0.0034, 0.0029, 0.0024, 0.0033],
         [0.0031, 0.0034, 0.0031, 0.0028, 0.0034]],

        [[0.0021, 0.0048, 0.0016, 0.0013, 0.0038],
         [0.0021, 0.0031, 0.0022, 0.0014, 0.0028],
         [0.0022, 0.0031, 0.0023, 0.0015, 0.0028],
         ...,
         [0.0026, 0.0034, 0.0027, 0.0020, 0.0032],
         [0.0028, 0.0034, 0.0029, 0.0024, 0.0033],
         [0.0031, 0.0034, 0.0031, 0.0028, 0.0034]],

        ...,

        [[0.0038, 0.